# Data cleaning

### Librairies

In [97]:
import pandas as pd
from functools import reduce
import re
import numpy as np

### import Data

In [98]:
df_cit = pd.read_csv('data/nb_cit_cleaning.csv')
df_downl = pd.read_csv('data/nb_downloads.csv')
df_pages = pd.read_csv('data/nb_pages.csv')
df_stu = pd.read_csv('data/nb_students.csv')
df_works = pd.read_csv('data/nb_works.csv')
df_hind = pd.read_csv('data/h_index.csv')
df_clark = pd.read_csv('data/clark_medal.csv')
df_vn = pd.read_csv('data/von_neumann.csv')
df_nobel = pd.read_csv('data/nobel_prize_winners.csv')

dfs = [df_downl, df_pages, df_stu, df_works, df_hind]

In [99]:
df_cit.head()

,Rank,Author,Score,Author_clean
0,1,"Andrei Shleifer Department of Economics, Harva...",48826,Andrei Shleifer
1,2,"James J. Heckman Department of Economics, Univ...",34146,James J. Heckman
2,3,"Robert J. Barro Department of Economics, Harva...",31421,Robert J. Barro
3,4,"Robert F. Engle IIIFinance Department, Stern S...",28370,Robert F. Engle
4,5,"Eugene F. Fama Sr.Booth School of Business, Un...",27760,Eugene F. Fama


In [100]:
# cleaning Author columns
for df in dfs:
    df['Author'] = df['Author'].apply(lambda x: x.replace('  ', ' '))
    df['Author_clean'] = df['Author'].apply(lambda x: x.split(' ',3)[0:3] if '.' in x else x.split(' ',2)[0:2])
    df['Author_clean'] = df['Author_clean'].apply(lambda x: ' '.join(x))
    df.drop('Author', axis=1, inplace=True)

In [101]:
df_downl.head()

,Rank,Score,Author_clean
0,1,7515,Christopher F
1,2,7462,Jeffrey Marc
2,3,6565,Ben Jann
3,4,6537,Daron Acemoglu
4,5,6424,Nicholas Cox


In [102]:
df_works.head()

,Rank,Score,Author_clean
0,1,1656,Simplice A
1,2,1383,Michael McAleer
2,3,1320,Peter Nijkamp
3,4,1270,Rangan Gupta
4,5,940,Pierre Pestieau


In [103]:
# renaming dataframes columns:
df_col_names = ['nb_downl', 'nb_pages', 'nb_students', 'nb_works', 'h_index']


In [104]:
for df, col in zip(dfs,df_col_names):
    df.rename(columns={'Score': col}, inplace=True)
df_cit.rename(columns={'Score':'nb_cit'}, inplace=True)

In [105]:
# process for merging my dataframes

# 1: add a Surname column in each df

# 2: remove duplicates in each df

# 3: merge them on "Surname"

# 4: count how many nobel prize winners are left in the final dataframe

In [106]:
# 1:
df_cit['Surname'] = df_cit['Author_clean'].str.split(' ')
df_cit['Surname'] = df_cit['Surname'].str[-1]
df_cit['First'] = df_cit['Author_clean'].str.split(' ').str[0]

for df in dfs:
    df['Surname'] = df['Author_clean'].str.split(' ')
    df['Surname'] = df['Surname'].str[-1]
    df['First'] = df['Author_clean'].str.split(' ').str[0]
df_cit.shape

(2944, 6)

In [107]:
# 2:
df_cit = df_cit.drop_duplicates(subset=['Surname', 'First'], keep='first' ) 
# checking there's no duplicates left
sum(df_cit.pivot_table(index=['Surname'], aggfunc='size') > 1)

for df in dfs:
    df = df.drop_duplicates(subset=['Surname', 'First'], keep='first')
    print('nb_duplicates :',sum(df.pivot_table(index=['Surname'], aggfunc='size') > 1))
    print('shapes :', df.shape)


nb_duplicates : 203
shapes : (2934, 5)
nb_duplicates : 234
shapes : (2946, 5)
nb_duplicates : 235
shapes : (2928, 6)
nb_duplicates : 212
shapes : (2965, 5)
nb_duplicates : 278
shapes : (3402, 5)


In [108]:
# 3:
df_merged = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, on=['Surname', 'First'],
                                              how='inner'), dfs)
df_merged2 = df_merged.drop_duplicates(subset=['Surname', 'First'], keep='first')

In [109]:
df_merged2.shape


(669, 18)

In [110]:
df_merged2.head()

,Rank_x,nb_downl,Author_clean_x,Surname,First,Rank_y,nb_pages,Author_clean_y,Rank_x,nb_students,Students,Author_clean_x,Rank_y,nb_works,Author_clean_y,Rank,h_index,Author_clean
0,1,7515,Christopher F,F,Christopher,1367,1099,Christopher F,328,374.56,10.0,Christopher F,95,415,Christopher F,1063,21,Christopher F
1,2,7462,Jeffrey Marc,Marc,Jeffrey,1440,1079,Jeffrey Marc,974,1037.32,4.0,Jeffrey Marc,749,214,Jeffrey Marc,299,31,Jeffrey Marc
2,4,6537,Daron Acemoglu,Acemoglu,Daron,10,5349,Daron Acemoglu,28,36.10,23.0,Daron Acemoglu,14,645,Daron Acemoglu,5,72,Daron Acemoglu
3,6,6290,Joseph E. Stiglitz,Stiglitz,Joseph,7,5753,Joseph E. Stiglitz,84,89.12,6.0,Joseph E. Stiglitz,17,643,Joseph E. Stiglitz,5,72,Joseph E. Stiglitz
4,7,5632,John List,List,John,33,3642,John List,349,387.06,5.0,John List,10,681,John List,19,56,John List


In [111]:
# add nber citations
df_result = pd.merge(df_merged2, df_cit, on=["Surname","First"],how="inner")

In [112]:
df_result.shape

(641, 22)

In [113]:
df_result.head()

,Rank_x,nb_downl,Author_clean_x,Surname,First,Rank_y,nb_pages,Author_clean_y,Rank_x,nb_students,...,Rank_y,nb_works,Author_clean_y,Rank_x,h_index,Author_clean_x,Rank_y,Author,nb_cit,Author_clean_y
0,1,7515,Christopher F,F,Christopher,1367,1099,Christopher F,328,374.56,...,95,415,Christopher F,1063,21,Christopher F,699,"Christopher F Baum Department of Economics, Bo...",3062,Christopher F
1,2,7462,Jeffrey Marc,Marc,Jeffrey,1440,1079,Jeffrey Marc,974,1037.32,...,749,214,Jeffrey Marc,299,31,Jeffrey Marc,83,"Jeffrey Marc Wooldridge Economics Department, ...",10615,Jeffrey Marc
2,4,6537,Daron Acemoglu,Acemoglu,Daron,10,5349,Daron Acemoglu,28,36.10,...,14,645,Daron Acemoglu,5,72,Daron Acemoglu,9,"Daron Acemoglu Economics Department, Massachus...",24703,Daron Acemoglu
3,6,6290,Joseph E. Stiglitz,Stiglitz,Joseph,7,5753,Joseph E. Stiglitz,84,89.12,...,17,643,Joseph E. Stiglitz,5,72,Joseph E. Stiglitz,7,Joseph E. Stiglitz Finance and Economics Depar...,25947,Joseph E. Stiglitz
4,7,5632,John List,List,John,33,3642,John List,349,387.06,...,10,681,John List,19,56,John List,74,"John List Department of Economics, University ...",11165,John List


In [117]:
# 4:
df_nobel['First'] = df_nobel['Author'].str.split(' ').str[0]
df_nobel['Surname'] = df_nobel['Author'].str.split(' ').str[-1]
print('nber of nobel prize winners: ', sum(df_result['Surname'].isin(df_nobel['Surname'])))
print('percentage of nobel prize winners in data:', sum(df_result['Surname'].isin(df_nobel['Surname']))/df_result.shape[0])

nber of nobel prize winners:  37
percentage of nobel prize winners in data: 0.057722308892355696


In [118]:
# Next steps:

# TODO: cleaning df_result
# TODO: adding columns for clark medal and von neumann
# TODO: add columns nobel prize winners
# TODO: feature engeneere to get the affiliate university & the country of the economist
# TODO: reduce to size of our data to 300 economists
# TODO: clean the jupyter notebook

In [119]:
# TODO: cleaning df_result
df_result.tail()

,Rank_x,nb_downl,Author_clean_x,Surname,First,Rank_y,nb_pages,Author_clean_y,Rank_x,nb_students,...,Rank_y,nb_works,Author_clean_y,Rank_x,h_index,Author_clean_x,Rank_y,Author,nb_cit,Author_clean_y
636,2881,363,Fabien Postel-Vinay,Postel-Vinay,Fabien,1809,964,Fabien Postel-Vinay,1722,1814.29,...,1770,143,Fabien Postel-Vinay,1403,19,Fabien Postel-Vinay,1424,"Fabien Postel-Vinay Department of Economics, U...",1766,Fabien Postel-Vinay
637,2896,362,Martin Paldam,Paldam,Martin,1729,985,Martin Paldam,720,764.92,...,2935,111,Martin Paldam,1403,19,Martin Paldam,1820,Martin Paldam School of Economics and Manageme...,1435,Martin Paldam
638,2896,362,Michael Christopher,Christopher,Michael,2400,840,Michael Christopher,218,240.06,...,903,197,Michael Christopher,1403,19,Michael Christopher,1877,Michael Christopher Burda Institut für Wirtsch...,1398,Michael Christopher
639,2909,361,Alan M. Rugman,Rugman,Alan,1979,920,Alan M. Rugman,2301,2390.19,...,2648,117,Alan M. Rugman,1063,21,Alan M. Rugman,1669,Alan M. Rugman †,1548,Alan M. Rugman
640,2924,360,Thomas R. Palfrey,Palfrey,Thomas,283,1995,Thomas R. Palfrey,1382,1473.91,...,519,247,Thomas R. Palfrey,299,31,Thomas R. Palfrey,500,"Thomas R. Palfrey Division of Social Sciences,...",3725,Thomas R. Palfrey


In [120]:
df_result = df_result.drop(['Rank_x', 'Author_clean_x','Rank_y'], axis=1)
df_result.columns

Index(['nb_downl', 'Surname', 'First', 'nb_pages', 'Author_clean_y',
       'nb_students', 'Students', 'nb_works', 'Author_clean_y', 'h_index',
       'Author', 'nb_cit', 'Author_clean_y'],
      dtype='object')

In [121]:
df_result = df_result.loc[:, ~df_result.columns.duplicated()]

In [122]:
df_result.drop(columns=['nb_students'], axis=1, inplace=True)

In [123]:
df_result.tail()

,nb_downl,Surname,First,nb_pages,Author_clean_y,Students,nb_works,h_index,Author,nb_cit
636,363,Postel-Vinay,Fabien,964,Fabien Postel-Vinay,3.0,143,19,"Fabien Postel-Vinay Department of Economics, U...",1766
637,362,Paldam,Martin,985,Martin Paldam,1.0,111,19,Martin Paldam School of Economics and Manageme...,1435
638,362,Christopher,Michael,840,Michael Christopher,15.0,197,19,Michael Christopher Burda Institut für Wirtsch...,1398
639,361,Rugman,Alan,920,Alan M. Rugman,0.5,117,21,Alan M. Rugman †,1548
640,360,Palfrey,Thomas,1995,Thomas R. Palfrey,1.0,247,31,"Thomas R. Palfrey Division of Social Sciences,...",3725


In [124]:
df_result.rename(columns={'Author_clean_y': 'Author_clean'}, inplace=True)
df_result.columns

Index(['nb_downl', 'Surname', 'First', 'nb_pages', 'Author_clean', 'Students',
       'nb_works', 'h_index', 'Author', 'nb_cit'],
      dtype='object')

In [125]:
df_result.isnull().any()

nb_downl        False
Surname         False
First           False
nb_pages        False
Author_clean    False
Students        False
nb_works        False
h_index         False
Author          False
nb_cit          False
dtype: bool

In [126]:
# TODO: adding columns for clark medal and von neumann
# cleaning clark dataset
ls_clark = list(df_clark['Medalists'])
ls_clark_new = []
for x in ls_clark:
    new_str = re.sub('[0-9]','',x)
    new_str = new_str.replace('[]','')
    ls_clark_new.append(new_str)
df_clark['Medalists'] = ls_clark_new
df_clark['Medalists'].tail()

37    Yuliy Sannikov
38    Dave Donaldson
39      Parag Pathak
40      Emi Nakamura
41      Melissa Dell
Name: Medalists, dtype: object

In [127]:
df_clark['Surname'] = df_clark['Medalists'].str.split(' ').str[-1]
df_clark['First'] = df_clark['Medalists'].str.split(' ').str[0]
df_clark.head()

,Year,Medalists,Institution (at time of receipt),Alma mater (PhD),Nationality,Nobel Prize,Surname,First
0,1947,Paul Samuelson,Massachusetts Institute of Technology,Harvard University,United States,1970.0,Samuelson,Paul
1,1949,Kenneth E. Boulding,University of Michigan,University of Oxford,United States,NaN,Boulding,Kenneth
2,1951,Milton Friedman,University of Chicago,Columbia University,United States,1976.0,Friedman,Milton
3,1955,James Tobin,Yale University,Harvard University,United States,1981.0,Tobin,James
4,1957,Kenneth Arrow,Stanford University,Columbia University,United States,1972.0,Arrow,Kenneth


In [128]:
# building boolean for clark
df_result_clark = df_result.copy()

df_boolclark = (df_result_clark['Author_clean'].isin(df_clark['Medalists'])) \
             | (df_result_clark['First'].isin(df_clark['First']) & df_result_clark['Surname'].isin(df_clark['Surname']))

print('number of clark medal winners in our dataset:',df_boolclark.sum())
print('percentage of clark medal winners present in our dataset:',round(df_boolclark.sum() / df_clark.shape[0], 2))                                
df_clark.shape

number of clark medal winners in our dataset: 23
percentage of clark medal winners present in our dataset: 0.55


(42, 8)

In [129]:
# cleaning von neuman dataset
ls_vn = list(df_vn['Recipient'])
ls_vn_new = []
for x in ls_vn:
    new_str = re.sub('[0-9]','',x)
    new_str = new_str.replace('[]','')
    ls_vn_new.append(new_str)
df_vn['Recipient'] = ls_vn_new
df_vn['Recipient'].tail()

20    Matthew O. Jackson
21         Alvin E. Roth
22     Richard H. Thaler
23           Dani Rodrik
24           Susan Athey
Name: Recipient, dtype: object

In [130]:
# building boolean for von neumann
df_vn['Surname'] = df_vn['Recipient'].str.split('.')
df_vn['Surname'] = df_vn['Recipient'].str.strip().str.split(' ')
df_vn['Surname'] = df_vn['Surname'].str[-1]

df_vn['First'] = df_vn['Recipient'].str.split(' ').str[0]
df_vn.head()

,Year,Recipient,Institution,Nationality,Surname,First
0,1995,John Harsanyi,UC Berkeley,United States,Harsanyi,John
1,1996,Hal Varian,University of Michigan,United States,Varian,Hal
2,1997,János Kornai,Harvard University; Collegium Budapest,Hungary,Kornai,János
3,1998,Jean Tirole,Toulouse School of Economics,France,Tirole,Jean
4,1999,Oliver E. Williamson,UC Berkeley,United States,Williamson,Oliver


In [131]:
df_result_vn = df_result.copy()

df_boolvn = (df_result_vn['Author_clean'].isin(df_vn['Recipient'])) \
             | (df_result_vn['First'].isin(df_vn['First']) & df_result_vn['Surname'].isin(df_vn['Surname']))

print('number of von neumann awards winners in our dataset:',df_boolvn.sum())
print('percentage of von neumann awards winners present in our dataset:',round(df_boolvn.sum() / df_vn.shape[0], 2))                                
df_boolvn.shape

number of von neumann awards winners in our dataset: 12
percentage of von neumann awards winners present in our dataset: 0.48


(641,)

In [132]:
# add column von neuman award
df_result['vn_award'] = 0
df_result['vn_award'][df_boolvn] = 1

C:\Users\imran\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [133]:
df_result['vn_award'].value_counts()

0    629
1     12
Name: vn_award, dtype: int64

In [134]:
# add column clark metal
df_result['clark'] = 0
df_result['clark'][df_boolclark] = 1 

C:\Users\imran\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [135]:
df_result['clark'].value_counts()

0    618
1     23
Name: clark, dtype: int64

In [136]:
df_result.head()

,nb_downl,Surname,First,nb_pages,Author_clean,Students,nb_works,h_index,Author,nb_cit,vn_award,clark
0,7515,F,Christopher,1099,Christopher F,10.0,415,21,"Christopher F Baum Department of Economics, Bo...",3062,0,0
1,7462,Marc,Jeffrey,1079,Jeffrey Marc,4.0,214,31,"Jeffrey Marc Wooldridge Economics Department, ...",10615,0,0
2,6537,Acemoglu,Daron,5349,Daron Acemoglu,23.0,645,72,"Daron Acemoglu Economics Department, Massachus...",24703,1,1
3,6290,Stiglitz,Joseph,5753,Joseph E. Stiglitz,6.0,643,72,Joseph E. Stiglitz Finance and Economics Depar...,25947,0,1
4,5632,List,John,3642,John List,5.0,681,56,"John List Department of Economics, University ...",11165,0,0


In [137]:
# TODO: add columns nobel prize winners

In [138]:
df_result_nobel = df_result.copy()

df_boolnobel = (df_result_nobel['Author_clean'].isin(df_nobel['Author'])) \
             | (df_result_nobel['First'].isin(df_nobel['First']) & df_result_nobel['Surname'].isin(df_nobel['Surname']))

print('number of nobel price winners in our dataset:',df_boolnobel.sum())
print('percentage of nobel price winners present in our dataset:',round(df_boolnobel.sum() / df_nobel.shape[0], 2))                                
df_boolnobel.shape

number of nobel price winners in our dataset: 30
percentage of nobel price winners present in our dataset: 0.36


(641,)

In [139]:
df_result['nobel'] = 0
df_result['nobel'][df_boolnobel] = 1
df_result[df_result['nobel'] == 1]

C:\Users\imran\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,nb_downl,Surname,First,nb_pages,Author_clean,Students,nb_works,h_index,Author,nb_cit,vn_award,clark,nobel
3,6290,Stiglitz,Joseph,5753,Joseph E. Stiglitz,6.0,643,72,Joseph E. Stiglitz Finance and Economics Depar...,25947,0,1,1
6,5097,Heckman,James,5018,James J. Heckman,26.0,645,86,"James J. Heckman Department of Economics, Univ...",34146,0,1,1
7,4891,Nordhaus,William,1793,William D. Nordhaus,0.5,208,34,William D. Nordhaus Cowles Foundation for Rese...,5083,0,0,1
9,3968,Sargent,Thomas,3439,Thomas J. Sargent,19.0,365,50,"Thomas J. Sargent Department of Economics, New...",9609,0,0,1
10,3956,Krugman,Paul,1636,Paul R. Krugman,8.5,199,49,Paul R. Krugman Woodrow Wilson School of Publi...,16124,0,1,1
12,3842,Becker,Gary,1323,Gary S. Becker,10.5,172,49,Gary S. Becker †,20728,1,1,1
21,3356,Lucas,Robert,1391,Robert E. Lucas,31.5,127,42,"Robert E. Lucas Jr.Department of Economics, Un...",18980,0,0,1
31,2924,Tirole,Jean,4439,Jean Tirole,5.0,478,74,Jean Tirole Institut d'Économie Industrielle (...,24483,1,0,1
35,2567,Engle,Robert,2116,Robert F. Engle,12.5,230,58,"Robert F. Engle IIIFinance Department, Stern S...",28370,0,0,1
40,2444,Tobin,James,1114,James Tobin,6.0,194,22,James Tobin †,3983,0,1,1


In [140]:
# TODO: feature engeneere to get the affiliate university & the country of the economist
for i in range(10,20):
    print(df_result['Author'].iloc[i])

Paul R. Krugman Woodrow Wilson School of Public and International Affairs, Princeton University, Princeton, New Jersey (USA)
Ross Levine National Bureau of Economic Research (NBER), Cambridge, Massachusetts (USA)
Gary S. Becker †
John Michael van Reenen Centre for Economic Performance (CEP), London School of Economics (LSE), London, United Kingdom
Ben S. Bernanke Economic Studies, Brookings Institution, Washington, District of Columbia (USA)
Barry Julian Eichengreen Department of Economics, University of California-Berkeley, Berkeley, California (USA)
Oded Galor Economics Department, Brown University, Providence, Rhode Island (USA)
Rafal Weron Katedra Badań Operacyjnych i Inteligencji Biznesowej, Politechnika Wrocławska, Wrocław, Poland
Dani Rodrik Kennedy School of Government, Harvard University, Cambridge, Massachusetts (USA)
Andrei Shleifer Department of Economics, Harvard University, Cambridge, Massachusetts (USA)National Bureau of Economic Research (NBER), Cambridge, Massachusetts

In [141]:
df_country = df_result['Author'].copy()
df_country_test = df_country.str.split(',').str[-1]

In [142]:
#df_country_test["USA" in df_country_test] = "United States"
df_country_test = df_country_test.apply(lambda x: x.split(' ')[-1] if "USA" in x else x)

In [143]:
#print(df_country_test.value_counts())
df_country_test2 = df_country_test.str.replace('.+†', 'None') 

In [144]:
df_country_test2.value_counts()
df_country_test2.rename('Country', axis=1, inplace=True)

0                (USA)
1                (USA)
2                (USA)
3                (USA)
4                (USA)
            ...       
636     United Kingdom
637            Denmark
638            Germany
639               None
640              (USA)
Name: Country, Length: 641, dtype: object

In [145]:
df_result = pd.concat([df_result, df_country_test2], axis=1)

In [146]:
df_result.head()

,nb_downl,Surname,First,nb_pages,Author_clean,Students,nb_works,h_index,Author,nb_cit,vn_award,clark,nobel,Country
0,7515,F,Christopher,1099,Christopher F,10.0,415,21,"Christopher F Baum Department of Economics, Bo...",3062,0,0,0,(USA)
1,7462,Marc,Jeffrey,1079,Jeffrey Marc,4.0,214,31,"Jeffrey Marc Wooldridge Economics Department, ...",10615,0,0,0,(USA)
2,6537,Acemoglu,Daron,5349,Daron Acemoglu,23.0,645,72,"Daron Acemoglu Economics Department, Massachus...",24703,1,1,0,(USA)
3,6290,Stiglitz,Joseph,5753,Joseph E. Stiglitz,6.0,643,72,Joseph E. Stiglitz Finance and Economics Depar...,25947,0,1,1,(USA)
4,5632,List,John,3642,John List,5.0,681,56,"John List Department of Economics, University ...",11165,0,0,0,(USA)


In [147]:
# TODO: feature engeneere to get the affiliate university of the economist
top10_shangai = ['University of Chicago', 'Harvard University', 'Massachusetts Institute of Technology',
                 'University of California-Berkeley','Princeton University', 'Columbia University',
                 'Stanford University', 'New York University','London School of Economics', 'Yale University']
#df_result_uni = df_result.copy()
df_result['top10_shangai_yn'] = df_result['Author'].apply(lambda x: 1 if any(uni in x for uni in top10_shangai) else 0)
df_result.head(10)

,nb_downl,Surname,First,nb_pages,Author_clean,Students,nb_works,h_index,Author,nb_cit,vn_award,clark,nobel,Country,top10_shangai_yn
0,7515,F,Christopher,1099,Christopher F,10.0,415,21,"Christopher F Baum Department of Economics, Bo...",3062,0,0,0,(USA),0
1,7462,Marc,Jeffrey,1079,Jeffrey Marc,4.0,214,31,"Jeffrey Marc Wooldridge Economics Department, ...",10615,0,0,0,(USA),0
2,6537,Acemoglu,Daron,5349,Daron Acemoglu,23.0,645,72,"Daron Acemoglu Economics Department, Massachus...",24703,1,1,0,(USA),1
3,6290,Stiglitz,Joseph,5753,Joseph E. Stiglitz,6.0,643,72,Joseph E. Stiglitz Finance and Economics Depar...,25947,0,1,1,(USA),1
4,5632,List,John,3642,John List,5.0,681,56,"John List Department of Economics, University ...",11165,0,0,0,(USA),1
5,5426,Barro,Robert,2705,Robert J. Barro,7.0,329,65,"Robert J. Barro Department of Economics, Harva...",31421,0,0,0,(USA),1
6,5097,Heckman,James,5018,James J. Heckman,26.0,645,86,"James J. Heckman Department of Economics, Univ...",34146,0,1,1,(USA),1
7,4891,Nordhaus,William,1793,William D. Nordhaus,0.5,208,34,William D. Nordhaus Cowles Foundation for Rese...,5083,0,0,1,(USA),1
8,4525,Hashem,M,3607,M Hashem,4.0,649,57,"M Hashem Pesaran Department of Economics, Univ...",23850,0,0,0,(USA),0
9,3968,Sargent,Thomas,3439,Thomas J. Sargent,19.0,365,50,"Thomas J. Sargent Department of Economics, New...",9609,0,0,1,(USA),1


In [148]:
for i in range(10):
    print(df_result['Author'].iloc[i])

Christopher F Baum Department of Economics, Boston College, Chestnut Hill, Massachusetts (USA)
Jeffrey Marc Wooldridge Economics Department, Michigan State University, East Lansing, Michigan (USA)
Daron Acemoglu Economics Department, Massachusetts Institute of Technology (MIT), Cambridge, Massachusetts (USA)
Joseph E. Stiglitz Finance and Economics Department, Graduate School of Business, Columbia University, New York City, New York (USA)
John List Department of Economics, University of Chicago, Chicago, Illinois (USA)
Robert J. Barro Department of Economics, Harvard University, Cambridge, Massachusetts (USA)
James J. Heckman Department of Economics, University of Chicago, Chicago, Illinois (USA)
William D. Nordhaus Cowles Foundation for Research in Economics, Yale University, New Haven, Connecticut (USA)Economics Department, Yale University, New Haven, Connecticut (USA)
M Hashem Pesaran Department of Economics, University of Southern California, Los Angeles, California (USA)
Thomas J.

In [149]:
df_result['top10_shangai_yn'].value_counts()

0    466
1    175
Name: top10_shangai_yn, dtype: int64

In [160]:
# last cleaning step : manually complete the affiliate university plus country of dead nobel winners economists (9)
# list of economists :
# Gary S. Becker, James Tobin, Kenneth J. Arrow, Milton Friedman, Paul A. Samuelson, Franco Modigliani, James M. Buchanan
# Paul W. Miller, Reinhard Selten

# correct country
nobel_names = ['Gary S. Becker', 'James Tobin', 'Kenneth J. Arrow', 'Milton Friedman', 'Paul A. Samuelson',
                    'Franco Modigliani', 'James M. Buchanan', 'Paul W. Miller', 'Reinhard Selten']
nobel_countries = ['(USA)', '(USA)', '(USA)', '(USA)', '(USA)', '(USA)', 
                   'Italy', '(USA)', 'Germany' ]
nobel_uni = [1, 1, 1, 1, 1,
             1, 0, 0, 0]
for name, ctry, uni in zip(nobel_names, nobel_countries, nobel_uni):
    df_result['Country'][df_result['Author_clean'] == name] = ctry
    df_result['top10_shangai_yn'][df_result['Author_clean'] == name] = uni
df_result[df_result['nobel'] == 1]

C:\Users\imran\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\imran\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,nb_downl,Surname,First,nb_pages,Author_clean,Students,nb_works,h_index,Author,nb_cit,vn_award,clark,nobel,Country,top10_shangai_yn
3,6290,Stiglitz,Joseph,5753,Joseph E. Stiglitz,6.0,643,72,Joseph E. Stiglitz Finance and Economics Depar...,25947,0,1,1,(USA),1
6,5097,Heckman,James,5018,James J. Heckman,26.0,645,86,"James J. Heckman Department of Economics, Univ...",34146,0,1,1,(USA),1
7,4891,Nordhaus,William,1793,William D. Nordhaus,0.5,208,34,William D. Nordhaus Cowles Foundation for Rese...,5083,0,0,1,(USA),1
9,3968,Sargent,Thomas,3439,Thomas J. Sargent,19.0,365,50,"Thomas J. Sargent Department of Economics, New...",9609,0,0,1,(USA),1
10,3956,Krugman,Paul,1636,Paul R. Krugman,8.5,199,49,Paul R. Krugman Woodrow Wilson School of Publi...,16124,0,1,1,(USA),1
12,3842,Becker,Gary,1323,Gary S. Becker,10.5,172,49,Gary S. Becker †,20728,1,1,1,(USA),1
21,3356,Lucas,Robert,1391,Robert E. Lucas,31.5,127,42,"Robert E. Lucas Jr.Department of Economics, Un...",18980,0,0,1,(USA),1
31,2924,Tirole,Jean,4439,Jean Tirole,5.0,478,74,Jean Tirole Institut d'Économie Industrielle (...,24483,1,0,1,France,0
35,2567,Engle,Robert,2116,Robert F. Engle,12.5,230,58,"Robert F. Engle IIIFinance Department, Stern S...",28370,0,0,1,(USA),1
40,2444,Tobin,James,1114,James Tobin,6.0,194,22,James Tobin †,3983,0,1,1,(USA),1


In [150]:
# TODO: Reduce the size of the dataset to 300 rows

In [161]:
# subest a dataframe with no nobel winners so we can randomly remove rows in it 
df_no_nobel = df_result[df_result['nobel'] == 0]
df_no_nobel.shape

(611, 15)

In [162]:
print('we want to remove {} rows' .format(df_result.shape[0] - 400))

we want to remove 241 rows


In [163]:
np.random.seed(10)

remove_n = 241
drop_indices = np.random.choice(df_no_nobel.index, remove_n, replace=False)
df_subset = df_result.drop(drop_indices)

In [164]:
df_subset['nobel'].value_counts()

0    370
1     30
Name: nobel, dtype: int64

In [165]:
# saving data cleaned to csv files
df_result.to_csv('data/data_cleaned.csv', index=False)
df_subset.to_csv('data/working_data.csv', index=False)